<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/ModelosLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo LSTM 70-30

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

url1 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data1 = pd.read_csv(url1)
data1 = data1.drop('Unnamed: 0',axis=1)
X = data1.drop(columns=['Inputs']).values
y = data1['Inputs'].values


# Escala los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Divide los datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

# Reestructura los datos para LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Construye y entrena el modelo
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Guarda el modelo y los escaladores
model.save('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/lstmFunc30.h5')
joblib.dump(scaler_X, '/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_X30.pkl')
joblib.dump(scaler_y, '/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_y30.pkl')

Epoch 1/10
9569/9569 - 38s - loss: 0.0090 - 38s/epoch - 4ms/step
Epoch 2/10
9569/9569 - 34s - loss: 0.0072 - 34s/epoch - 4ms/step
Epoch 3/10
9569/9569 - 35s - loss: 0.0064 - 35s/epoch - 4ms/step
Epoch 4/10
9569/9569 - 33s - loss: 0.0062 - 33s/epoch - 3ms/step
Epoch 5/10
9569/9569 - 33s - loss: 0.0061 - 33s/epoch - 3ms/step
Epoch 6/10
9569/9569 - 34s - loss: 0.0059 - 34s/epoch - 4ms/step
Epoch 7/10
9569/9569 - 33s - loss: 0.0059 - 33s/epoch - 3ms/step
Epoch 8/10
9569/9569 - 34s - loss: 0.0058 - 34s/epoch - 4ms/step
Epoch 9/10
9569/9569 - 33s - loss: 0.0058 - 33s/epoch - 3ms/step
Epoch 10/10
9569/9569 - 33s - loss: 0.0058 - 33s/epoch - 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_y30.pkl']

In [12]:
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Carga el modelo y los escaladores
model = load_model('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/lstmFunc30.h5')
scaler_X = joblib.load('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_X30.pkl')
scaler_y = joblib.load('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_y30.pkl')

def make_prediction(new_data):
    new_data_scaled = scaler_X.transform(new_data)
    new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
    prediction_scaled = model.predict(new_data_scaled)
    prediction = scaler_y.inverse_transform(prediction_scaled)
    return prediction

# Lista de estaciones
stations = [
    "Station_(02000)Cabecera Autopista Norte",
    "Station_(02300)Calle 100",
    "Station_(03000)Portal Suba",
    "Station_(04000)Cabecera Calle 80",
    "Station_(05000)Portal Américas",
    "Station_(06000)Portal Eldorado",
    "Station_(07000)Portal Sur",
    "Station_(08000)Portal Tunal",
    "Station_(09000)Cabecera Usme",
    "Station_(10000)Portal 20 de Julio"
]

# Mostrar las estaciones al usuario
print("Selecciona una estación ingresando el número correspondiente:")
for i, station in enumerate(stations, 1):
    print(f"{i} {station}")

# Leer la selección del usuario
station_index = int(input("Número de estación seleccionada: ")) - 1

# Solicitar al usuario el resto de los valores
print("Ingresa los valores para Month, Day, Day_Week_Number, Holidays, Hour, Minute, Outputs (en ese orden), separados por comas:")
input_str = input()  # El usuario ingresa los valores separados por comas
values = [float(value) for value in input_str.split(',')]

# Asegurar que se ingresaron 7 valores antes de proceder
if len(values) != 7:
    raise ValueError("Debes ingresar exactamente 7 valores.")

# Construir el array new_data
new_data_values = values + [0.0] * 10  # Inicializa los valores de las estaciones con 0.0
new_data_values[7 + station_index] = 1.0  # Establece la estación seleccionada a 1
new_data = np.array([new_data_values])

# Hacer la predicción
prediction = make_prediction(new_data)
print("La predicción para 'Inputs' es:", prediction.flatten()[0])


Selecciona una estación ingresando el número correspondiente:
1 Station_(02000)Cabecera Autopista Norte
2 Station_(02300)Calle 100
3 Station_(03000)Portal Suba
4 Station_(04000)Cabecera Calle 80
5 Station_(05000)Portal Américas
6 Station_(06000)Portal Eldorado
7 Station_(07000)Portal Sur
8 Station_(08000)Portal Tunal
9 Station_(09000)Cabecera Usme
10 Station_(10000)Portal 20 de Julio
Número de estación seleccionada: 3
Ingresa los valores para Month, Day, Day_Week_Number, Holidays, Hour, Minute, Outputs (en ese orden), separados por comas:
12, 4, 1, 0, 10, 15, 42
1/1 [==============================] - 1s 725ms/step
La predicción para 'Inputs' es: 7.4791074


# Modelo 80-20


In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

url1 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data1 = pd.read_csv(url1)
data1 = data1.drop('Unnamed: 0',axis=1)
X = data1.drop(columns=['Inputs']).values
y = data1['Inputs'].values


# Escala los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Divide los datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)

# Reestructura los datos para LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Construye y entrena el modelo
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Guarda el modelo y los escaladores
model.save('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/lstmFunc20.h5')
joblib.dump(scaler_X, '/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_X20.pkl')
joblib.dump(scaler_y, '/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_y20.pkl')

Epoch 1/10
9569/9569 - 39s - loss: 0.0089 - 39s/epoch - 4ms/step
Epoch 2/10
9569/9569 - 36s - loss: 0.0070 - 36s/epoch - 4ms/step
Epoch 3/10
9569/9569 - 37s - loss: 0.0063 - 37s/epoch - 4ms/step
Epoch 4/10
9569/9569 - 36s - loss: 0.0062 - 36s/epoch - 4ms/step
Epoch 5/10
9569/9569 - 35s - loss: 0.0060 - 35s/epoch - 4ms/step
Epoch 6/10
9569/9569 - 34s - loss: 0.0059 - 34s/epoch - 4ms/step
Epoch 7/10
9569/9569 - 35s - loss: 0.0059 - 35s/epoch - 4ms/step
Epoch 8/10
9569/9569 - 35s - loss: 0.0058 - 35s/epoch - 4ms/step
Epoch 9/10
9569/9569 - 36s - loss: 0.0058 - 36s/epoch - 4ms/step
Epoch 10/10
9569/9569 - 34s - loss: 0.0058 - 34s/epoch - 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_y20.pkl']

In [14]:
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Carga el modelo y los escaladores
model = load_model('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/lstmFunc20.h5')
scaler_X = joblib.load('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_X20.pkl')
scaler_y = joblib.load('/content/drive/MyDrive/Trabajo de Grado/Modelo LSTM/scaler_y20.pkl')

def make_prediction(new_data):
    new_data_scaled = scaler_X.transform(new_data)
    new_data_scaled = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
    prediction_scaled = model.predict(new_data_scaled)
    prediction = scaler_y.inverse_transform(prediction_scaled)
    return prediction

# Lista de estaciones
stations = [
    "Station_(02000)Cabecera Autopista Norte",
    "Station_(02300)Calle 100",
    "Station_(03000)Portal Suba",
    "Station_(04000)Cabecera Calle 80",
    "Station_(05000)Portal Américas",
    "Station_(06000)Portal Eldorado",
    "Station_(07000)Portal Sur",
    "Station_(08000)Portal Tunal",
    "Station_(09000)Cabecera Usme",
    "Station_(10000)Portal 20 de Julio"
]

# Mostrar las estaciones al usuario
print("Selecciona una estación ingresando el número correspondiente:")
for i, station in enumerate(stations, 1):
    print(f"{i} {station}")

# Leer la selección del usuario
station_index = int(input("Número de estación seleccionada: ")) - 1

# Solicitar al usuario el resto de los valores
print("Ingresa los valores para Month, Day, Day_Week_Number, Holidays, Hour, Minute, Outputs (en ese orden), separados por comas:")
input_str = input()  # El usuario ingresa los valores separados por comas
values = [float(value) for value in input_str.split(',')]

# Asegurar que se ingresaron 7 valores antes de proceder
if len(values) != 7:
    raise ValueError("Debes ingresar exactamente 7 valores.")

# Construir el array new_data
new_data_values = values + [0.0] * 10  # Inicializa los valores de las estaciones con 0.0
new_data_values[7 + station_index] = 1.0  # Establece la estación seleccionada a 1
new_data = np.array([new_data_values])

# Hacer la predicción
prediction = make_prediction(new_data)
print("La predicción para 'Inputs' es:", prediction.flatten()[0])

Selecciona una estación ingresando el número correspondiente:
1 Station_(02000)Cabecera Autopista Norte
2 Station_(02300)Calle 100
3 Station_(03000)Portal Suba
4 Station_(04000)Cabecera Calle 80
5 Station_(05000)Portal Américas
6 Station_(06000)Portal Eldorado
7 Station_(07000)Portal Sur
8 Station_(08000)Portal Tunal
9 Station_(09000)Cabecera Usme
10 Station_(10000)Portal 20 de Julio
Número de estación seleccionada: 3
Ingresa los valores para Month, Day, Day_Week_Number, Holidays, Hour, Minute, Outputs (en ese orden), separados por comas:
12, 4, 1, 0, 10, 15, 42
1/1 [==============================] - 1s 753ms/step
La predicción para 'Inputs' es: 2.888681
